In [1]:
%pip install pydantic_settings langchain langchain-core langchain-google-genai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### IVF Embryo Grading

In [2]:
import base64
import json

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate

In [ ]:
# Process the image to base64 so it can be sent to the LLM

image_path = "ivf_embryo_images/day2_grade1.png"

with open(image_path, "rb") as image_file:
    base64_image = base64.b64encode(image_file.read()).decode('utf-8')

In [4]:
# Load environment variables

from pydantic_settings import BaseSettings, SettingsConfigDict

class Settings(BaseSettings):
    GOOGLE_API_KEY: str
    model_config = SettingsConfigDict(env_file=".env")

env = Settings()

In [5]:
# Initialize the LLM with the Google Generative AI model

llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        api_key=env.GOOGLE_API_KEY,
    )

In [6]:
system_prompt = """
You are an expert AI assistant specializing in the analysis of human IVF embryo images. Your task is to analyze the provided microscopic image of a human embryo using a systematic chain-of-thought approach.

## CHAIN OF THOUGHT ANALYSIS PROCESS:

### Step 1: Initial Image Assessment
First, carefully examine the overall image quality and identify:
- Is this a clear microscopic image of an embryo?
- What is the overall shape and structure visible?
- Are there any obvious artifacts or issues with image quality?

### Step 2: Structural Feature Identification
Systematically identify the following key structures (note presence/absence):
- **Zona Pellucida**: The clear outer protective layer around the embryo
- **Cell boundaries**: Can individual cells (blastomeres) be distinguished?
- **Pronuclei (2PN)**: Two distinct circular structures within a single cell
- **Polar Body**: Small cellular structure in the perivitelline space
- **Blastocoel cavity**: A fluid-filled space within the embryo
- **Inner Cell Mass (ICM)**: A distinct cluster of cells (usually to one side)
- **Trophectoderm (TE)**: The outer cell layer forming the boundary
- **Fragmentation**: Small cellular debris or fragments

### Step 3: Developmental Stage Determination Logic
Follow this decision tree systematically:

**Question A**: Are there TWO distinct pronuclei (2PN) visible within a single large cell?
- If YES → This is **Day 1 (Zygote stage)**
- If NO → Continue to Question B

**Question B**: Are there multiple distinct cells (blastomeres) visible with clear boundaries?
- If NO (single large cell, no 2PN) → This is **Day 0 (Oocyte)**
- If YES → Continue to Question C

**Question C**: How many cells are visible and what is their arrangement?
- If 2-4 cells → Likely **Day 2**
- If 5-8 cells → Likely **Day 3**
- If many cells in compact mass → Continue to Question D

**Question D**: Is there a distinct fluid-filled cavity (blastocoel) AND clear differentiation between ICM and TE?
- If NO (solid compact mass, no cavity, no ICM/TE distinction) → This is **Day 4 (Morula)**
- If YES (cavity present + ICM/TE visible) → This is **Day 5/6 (Blastocyst)**

### Step 4: Quality Assessment Based on Stage
**For Day 0/1**: No grading applies - assess maturity indicators
**For Day 2-4 (Cleavage Stage)**: Evaluate using 1-5 scale:
- Cell size uniformity
- Fragmentation percentage (<10%, 10-25%, 25-50%, >50%)
- Cytoplasm appearance
- Overall morphology

**For Day 5-6 (Blastocyst)**: Use Gardner system (Number + Letter + Letter):
1. **Expansion (1-6)**: Assess blastocoel size and zona pellucida condition
2. **ICM Quality (A/B/C)**: Evaluate cell mass compactness and cell number
3. **TE Quality (A/B/C)**: Evaluate outer layer cohesiveness and organization

### Step 5: Final Verification
Double-check your assessment:
- Does the assigned day match the observed features?
- Are the grading criteria appropriately applied?
- Is there consistency between stage and grade assignments?

## CONTEXT AND GRADING SYSTEMS:

**Developmental Timeline & Morphology:**
- **Day 0**: Single oocyte, possibly with polar body, NO 2PN, NO cleavage
- **Day 1**: Zygote with two distinct pronuclei (2PN)
- **Day 2**: 2-4 cells (blastomeres)
- **Day 3**: 5-8 cells, relatively even
- **Day 4**: Morula - solid mass due to compaction, NO distinct cavity, NO ICM/TE differentiation
- **Day 5/6**: Blastocyst - MUST have BOTH distinct blastocoel AND visible ICM/TE differentiation

**Cleavage Stage Grading (Day 2-4)**: Scale 1-5
- Grade 1 (Excellent): Even blastomeres, <10% fragmentation, clear cytoplasm
- Grade 2 (Good): Slightly uneven, ≤10% fragmentation
- Grade 3 (Average): Uneven blastomeres OR 10-25% fragmentation
- Grade 4 (Below Average): Significant irregularities, 25-50% fragmentation
- Grade 5 (Poor): Severe irregularities, >50% fragmentation

**Blastocyst Grading (Day 5-6)**: Gardner System (e.g., 4AA, 3BC)
- **Expansion (1-6)**: 1=early cavity, 2=visible cavity, 3=full cavity, 4=expanded+thin zona, 5=hatching, 6=hatched
- **ICM Quality (A/B/C)**: A=many tightly packed cells, B=several loose cells, C=few/poor cells
- **TE Quality (A/B/C)**: A=many cohesive cells, B=fewer loose cells, C=very few/irregular cells

## OUTPUT FORMAT:
Provide your step-by-step analysis followed by the final assessment in JSON format:

```json
{{
  "day": integer,
  "cleavage_stage_grade": integer | null,
  "blastocyst_stage_grade": string | null,
  "for_doctor_explanation": "string",
  "for_patient_explanation": "string"
}}
```

**Grading Rules:**
- Day 0/1: Both grade fields = null
- Day 2-4: Only cleavage_stage_grade populated, blastocyst_stage_grade = null
- Day 5-6: Only blastocyst_stage_grade populated, cleavage_stage_grade = null

Now analyze the provided embryo image following this systematic chain-of-thought approach.
"""

In [7]:
# Create the prompt for the LLM 

prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", [
            {
                "type": "text",
                "text": "Analyze this IVF embryo image following the chain-of-thought approach."
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}"
                }
            }
        ])
    ])

In [8]:
# Create the chain to invoke the LLM

chain = prompt | llm
response = chain.invoke({})
response.content

'Okay, I will analyze the provided IVF embryo image using the defined chain-of-thought approach.\n\n**Step 1: Initial Image Assessment**\n- The image is a clear microscopic view of an embryo.\n- The overall shape is spherical, contained within the zona pellucida.\n- No obvious artifacts significantly impair the analysis.\n\n**Step 2: Structural Feature Identification**\n- **Zona Pellucida**: Present and clearly visible.\n- **Cell boundaries**: Four distinct cells (blastomeres) are visible, and their boundaries are relatively clear.\n- **Pronuclei (2PN)**: Not visible.\n- **Polar Body**: A polar body appears to be present.\n- **Blastocoel cavity**: Not visible.\n- **Inner Cell Mass (ICM)**: Not applicable at this stage.\n- **Trophectoderm (TE)**: Not applicable at this stage.\n- **Fragmentation**: Minimal to no visible fragmentation.\n\n**Step 3: Developmental Stage Determination Logic**\n\n*   **Question A**: Are there TWO distinct pronuclei (2PN) visible within a single large cell?\n 

In [14]:
def print_readable_result(response_text: str) -> None:
    """
    Print the analysis result in a readable format
    
    Args:
        response_text (str): Raw response from LLM
    """
    try:
        # Try to extract JSON from response
        json_start = response_text.find('{')
        json_end = response_text.rfind('}') + 1
        
        if json_start != -1 and json_end != -1:
            # Extract and parse JSON
            json_str = response_text[json_start:json_end]
            result = json.loads(json_str)
            
            # Print formatted result
            print("=" * 60)
            print("🔬 IVF EMBRYO ANALYSIS RESULT")
            print("=" * 60)
            
            print(f"📅 Developmental Day: {result.get('day', 'N/A')}")
            
            # Show grade based on stage
            if result.get('day') in [0, 1]:
                print("📊 Grade: No grading applicable")
            elif result.get('cleavage_stage_grade'):
                grade = result.get('cleavage_stage_grade')
                grade_names = {1: "Excellent", 2: "Good", 3: "Average", 4: "Below Average", 5: "Poor"}
                print(f"📊 Cleavage Grade: {grade}/5 ({grade_names.get(grade, 'Unknown')})")
            elif result.get('blastocyst_stage_grade'):
                print(f"📊 Blastocyst Grade: {result.get('blastocyst_stage_grade')}")
            
            print("\n📋 TECHNICAL EXPLANATION:")
            print("-" * 40)
            print(result.get('for_doctor_explanation', 'N/A'))
            
            print("\n👤 PATIENT EXPLANATION:")
            print("-" * 40)
            print(result.get('for_patient_explanation', 'N/A'))
            
            # Show chain of thought if available
            chain_of_thought = response_text[:json_start].strip()
            if chain_of_thought:
                print("\n🧠 ANALYSIS PROCESS:")
                print("-" * 40)
                print(chain_of_thought)
            
            print("=" * 60)
            
        else:
            # If no JSON found, just print the raw response
            print("🤖 RAW RESPONSE:")
            print("-" * 40)
            print(response_text)
            
    except Exception as e:
        print(f"❌ Error formatting result: {e}")
        print("\n🤖 RAW RESPONSE:")
        print("-" * 40)
        print(response_text)

In [15]:
print_readable_result(response.content)

🔬 IVF EMBRYO ANALYSIS RESULT
📅 Developmental Day: 2
📊 Cleavage Grade: 2/5 (Good)

📋 TECHNICAL EXPLANATION:
----------------------------------------
The embryo is at the 4-cell stage, consistent with Day 2 development. The blastomeres are relatively uniform in size, with minimal fragmentation. Overall morphology appears good.

👤 PATIENT EXPLANATION:
----------------------------------------
Your embryo has 4 cells and is developing as expected for day 2. The cells look even and there's very little fragmentation, which is a good sign.

🧠 ANALYSIS PROCESS:
----------------------------------------
Okay, I will analyze the provided IVF embryo image using the defined chain-of-thought approach.

**Step 1: Initial Image Assessment**
- The image is a clear microscopic view of an embryo.
- The overall shape is spherical, contained within the zona pellucida.
- No obvious artifacts significantly impair the analysis.

**Step 2: Structural Feature Identification**
- **Zona Pellucida**: Present and cl